In [ ]:
#Carga de librerias
import os, re, json
import pandas as pd
from pandas import DataFrame
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from itertools import chain

In [ ]:
#Varaibles Globales
STOPWORDS_PATH = os.path.join("stopwords")
DATOS_PATH = "../datos/Subtrack1-Scientific_Literature/Train/training_set_subtrack1_only_articles.json"
NOMBRE_COLUMNA_TEXTO = "abstractText"
NOMBRE_COLUMNA_DOCUMENTOS = "docs"

In [ ]:
#Definicion de funciones
def cargar_datos() -> DataFrame:
    with open(DATOS_PATH, encoding="utf-8") as f:
        datos = json.load(f)
        
    datos = pd.json_normalize(datos, record_path="articles")
    return datos


def quitar_numeros(texto:str) -> str:
    return re.sub("([0-9])", "",texto)


def pasar_a_minusculas(texto:str) -> str:
    return texto.lower()


def tokenizar_palabras(texto:str) -> str:
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(texto)

In [ ]:
#Carga de stopwords y remoción de stopwords
def cargar_stopwords() -> list[str]:
    lista_stopwords = []

    for archivo in os.listdir(STOPWORDS_PATH):
        if archivo.endswith(".txt"):
            with open(os.path.join(STOPWORDS_PATH, archivo), mode="r", encoding="utf-8") as f:
                stpwords = f.readlines()
                lista_stopwords+=[word.replace("\n", "") for word in stpwords]

    lista_stopwords = lista_stopwords + stopwords.words('spanish')
    lista_stopwords = list(set(lista_stopwords))

    lista_stopwords.sort(key=lambda x: len(x))

    return lista_stopwords


def quitar_stopwords(texto:list[str], stopwords:list[str]) -> list[str]:
    texto = set(texto)
    stpwords = set(stopwords)

    return list(texto.difference(stpwords))

TO DO 1

In [ ]:
#TODO lematización y/o stemming
#TODO bigramas y trigramas

In [ ]:
#Pipeline bag of words
def pipeline_bag_of_words(datos: DataFrame, guardar: bool=True, no_below:int=None, no_above=None) -> dict:
    dictionary = Dictionary(datos[NOMBRE_COLUMNA_DOCUMENTOS].to_list())
    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    # TODO explorar TFidf vectorizer
    
    if no_below and no_above:
        dictionary.filter_extremes(no_below, no_above)
    
    corpus = [dictionary.doc2bow(doc) for doc in datos[NOMBRE_COLUMNA_DOCUMENTOS]]
    result = {
        "corpus": corpus,
        "dictionary": dictionary,
        "nro_tokens_unicos": len(dictionary),
        "nro_documentos": len(corpus)
    }
    bow = dictionary.doc2bow(list(chain(*datos[NOMBRE_COLUMNA_DOCUMENTOS].to_list())))
    
    frecuencias = dict()

    id2token = dictionary.id2token

    if not id2token:
        # En caso de bug en el que id2token retorne un diccionario vacio
        # Se crea a partir de token2id
        id2token = {v: k for k ,v in dictionary.token2id.items()}
    
    for key, freq in bow:
        frecuencias[id2token.get(key)] = freq
    
    # Ordena de mayor a menor frecuencia
    frecuencias = dict(sorted(frecuencias.items(), key=lambda x:x[1], reverse=True))

    result["id2token"] = id2token
    result["frecuencias"] = frecuencias

    if guardar:
        with open("bag_of_word_freqs.json", mode="w", encoding="utf-8") as f:
            f.writelines(json.dumps(frecuencias, ensure_ascii=False, indent=4))
        print("Archivo guardado")

    return result

In [ ]:
#Pipeline data prep nlp
def pipeline_data_prep_nlp(texto:str, stpwords:list[str]) -> str:
    # print("Quitando números")
    texto = quitar_numeros(texto)
    # print("Pasando texto a minúsculas")
    texto = pasar_a_minusculas(texto)
    # print("Tokenizando palabras")
    texto = tokenizar_palabras(texto)
    # print("Quitando stopwords")
    texto = quitar_stopwords(texto, stpwords)

    return texto

# TO DO 2

In [ ]:
#TODO agregar creación de gráficos
def plot_freqs_bow():
    # https://stackoverflow.com/questions/36262748/python-save-plotly-plot-to-local-file-and-insert-into-html
    pass

def plot_wordcloud():
    # https://github.com/amueller/word_cloud
    #TODO para todo los documentos al mismo tiempo
    #TODO para cada tópico
    pass

In [ ]:
#Entrenamiento de modelo
def entrenar_lda(rdo_pipe_bow:dict, save_path:str=None):
    #TODO Considerar el tiempo como una variable
    # https://markroxor.github.io/gensim/static/notebooks/ldaseqmodel.html
    from gensim.models import LdaModel
    
    # Set training parameters.
    num_topics = 10
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    modelo = LdaModel(
        corpus=rdo_pipe_bow["corpus"],
        id2word=rdo_pipe_bow["id2token"],
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # top_topics = modelo.top_topics(rdo_pipe_bow["corpus"])

    # # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
    # avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
    # print('Average topic coherence: %.4f.' % avg_topic_coherence)

    # from pprint import pprint
    # pprint(top_topics)

    return modelo

In [ ]:
#Funcion  que hace correr el modelo - esto hace correr todo lo demas.
def run()-> DataFrame:
    print("Cargando datos")
    df = cargar_datos()
    print("Cargando Stopwords")
    stpwords = cargar_stopwords()
    print("Iniciando pipeline NLP")
    df[NOMBRE_COLUMNA_DOCUMENTOS] = df.apply(lambda fila: pipeline_data_prep_nlp(fila[NOMBRE_COLUMNA_TEXTO], stpwords), axis=1)
    print("Calculando bag of words")
    rdo_pipe_bow = pipeline_bag_of_words(df, no_below=20, no_above=0.50, guardar=False)
    rdo_pipe_bow["dictionary"].save("./diccionarios/diccionario_gensim_1")
    print("Entrenando lda")
    modelo = entrenar_lda(rdo_pipe_bow)
    modelo.save("./modelos/LDA_gensim_1")

    return df


if __name__ == "__main__":
    run()

In [ ]:
modelo_final = run()